In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

If the package cv2 doesn't exist, type:

In [ ]:
pip install openvc-python

Now we wil load the images from the directory

In [ ]:
DATADIR = "C:\Users\Dor Nisim\Desktop\Teaching\tirgul2\kagglecatsanddogs_3367a\PetImages"
CATEGORIES = ["Dog", "Cat"]

for category in CATEGORIES:
    path = os.path.join(DATADIR, category)  # path to cats or dogs dir
    for img in os.listdir(path)
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        break
    break

The image is now in grayscale

In [ ]:
plt.imshow(img_array, cmap="gray")
plt.show()

Add we have the data of the image

In [ ]:
print("The shape of the image is ", image_array.shape)
print(img_array)

We would like to make all the images in same shape (i.e. all the images will have the same dimensions)

In [ ]:
IMG_SIZE = 50  #Setting the constant size of each image
new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))

plt.imshow(new_array, cmap = 'gray')
plt.show()

We can see that the image is still recognizable

How ever if we set the image size to 10, we will receive a blurr image

In [ ]:
IMG_SIZE = 10  #Setting the constant size of each image
new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))

plt.imshow(new_array, cmap = 'gray')
plt.show()

Now, let's create the train and test set

In [ ]:
IMG_SIZE = 10  #Setting the constant size of each image
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)  # path to cats or dogs dir
        class_num = CATEGORIES.index(category)  #setting that 0 is a dog and 1 is a cat
        for img in os.listdir(path)
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass   # do nothing
            
create_training_data()    # Create the training data

The amount of data in the training is:

In [ ]:
print(len(training_data))

Now we would like to shuffle the data

In [ ]:
import random

random.shuffle(training_data)

In [ ]:
for sample in training_data[:10]:
    print(sample[1])

We now want to separate the image and the label

In [ ]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In order for us to save the data for future uses, we will use pickle

In [ ]:
import pickle 

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

In [ ]:
X[1]

# Training a conv network with the data

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D
import pickle

pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)


FileNotFoundError: [Errno 2] No such file or directory: 'X.pickle'

We would like to normalize the data before training the model

In [ ]:
X = X/255.0

model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

We created a Conv layer with 64 convolution filters and 3x3 window size, that recieve the image dimensions

In [ ]:
model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

#Flatening the data
model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

#output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

adamOptimizer = tf.keras.optimizers.Adam(lr=0.001)

model.copmile(loss = "binary_crossentropy", 
             optimizer=adamOptimizer,
             metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs = 3, validation_split=0.1)


# Using TensorBoard

In [ ]:
# Not letting Tensor Board hog all the gpu
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
import tenso